In [1]:
from lib import lib
import pandas as pd
from PIL import Image
import requests
from io import BytesIO
import os.path


In [46]:
lib = lib({
    'token': "insert your token here",
})
version = "@1.4.0"
gamertag = "valent 33"


In [47]:
def getAllMedals(gamertag, season=None):
    medals = pd.DataFrame(lib.halo.infinite[version].metadata.multiplayer.medals()['data'])
    medals['image_urls'] = medals['image_urls'].apply(lambda x: x['large'])
    medals['count'] = 0

    result = lib.halo.infinite[version].stats.players['service-record'].multiplayer.matchmade.all({
        'gamertag': gamertag,
        'season': season,
    })
    medal_count = pd.DataFrame(result['data']['core']['breakdowns']['medals'])
    for i in range(len(medal_count)):
        medals.loc[medals['id'] == medal_count['id'][i], 'count'] = medal_count['count'][i]
    
    return medals


In [49]:
medals = getAllMedals()


In [50]:
medals.head()


,id,name,description,difficulty,type,image_urls,count
0,3233952928,Killjoy,End an enemy's killing spree,normal,killing-spree,https://halo.public.files.stdlib.com/static/in...,163
1,2780740615,Killing Spree,Kill 5 enemies without dying,heroic,killing-spree,https://halo.public.files.stdlib.com/static/in...,649
2,4261842076,Killing Frenzy,Kill 10 enemies without dying,legendary,killing-spree,https://halo.public.files.stdlib.com/static/in...,194
3,418532952,Running Riot,Kill 15 enemies without dying,mythic,killing-spree,https://halo.public.files.stdlib.com/static/in...,49
4,1486797009,Rampage,Kill 20 enemies without dying,mythic,killing-spree,https://halo.public.files.stdlib.com/static/in...,17


In [ ]:
for i in range(len(medals)):
    response = requests.get(medals['image_urls'][i])
    img = Image.open(BytesIO(response.content))
    img.save('.\images\\' + medals['name'][i] + '.png')


In [41]:
path = ".\images"
dirs = os.listdir(path)


def crop():
    for item in dirs:
        if item not in ['Killing Spree.png']:
            fullpath = os.path.join(path, item)
            if os.path.isfile(fullpath):
                im = Image.open(fullpath)
                f, e = os.path.splitext(fullpath)
                imCrop = im.crop((20, 20, 236, 236))
                imCrop.save(f + 'Cropped.png', "png")


crop()


In [42]:
# delete all images but the cropped ones
for item in dirs:
    if item not in ['Killing Spree.png']:
        fullpath = os.path.join(path, item)
        if os.path.isfile(fullpath):
            f, e = os.path.splitext(fullpath)
            if f + 'Cropped.png' not in os.listdir(path):
                os.remove(fullpath)
    else:
        # rename the image by adding Cropped
        fullpath = os.path.join(path, item)
        f, e = os.path.splitext(fullpath)
        os.rename(fullpath, f + 'Cropped.png')


In [51]:
gephi = pd.DataFrame(columns=['source', 'weight', 'type', 'image'])
gephi['source'] = medals[medals['count'] != 0]['name']
gephi['weight'] = medals[medals['count'] != 0]['count']
gephi['type'] = medals[medals['count'] != 0]['type']
gephi['image'] = medals[medals['count'] != 0]['name']
gephi['image'] = gephi['image'].apply(lambda x: x + 'Cropped.png')


In [52]:
gephi.head()


,source,weight,type,image
0,Killjoy,163,killing-spree,KilljoyCropped.png
1,Killing Spree,649,killing-spree,Killing SpreeCropped.png
2,Killing Frenzy,194,killing-spree,Killing FrenzyCropped.png
3,Running Riot,49,killing-spree,Running RiotCropped.png
4,Rampage,17,killing-spree,RampageCropped.png


In [53]:
with open('halo.csv', 'w') as f:
    f.write('id')
gephi.to_csv('halo.csv', mode='a')
